In [17]:
#!pip install torch==1.4.0
#!pip install torchaudio==0.4.0
#!pip install syft==0.2.6

In [1]:
%%time
import torchaudio
import torch.optim as optim

import numpy as np
from glob import glob
from difflib import SequenceMatcher
import os
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.functional import F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
import librosa
import torch
import torch.nn as nn
from librosa.core import stft, magphase
from torch.autograd import Variable
## from data_load import CodeSwitchDataset

from syft.frameworks.torch.fl.utils import federated_avg

CPU times: user 1.98 s, sys: 160 ms, total: 2.14 s
Wall time: 2.96 s


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
class CodeSwitchDataset(Dataset):
    def __init__(self, lang, client, mode = "train", shuffle=True):
        self.mode = mode
        # data path
        self.lang = lang
        if self.lang == "Gujarati":
            self.max_len = 0
        elif self.lang == "Telugu":
            self.max_len = 529862
        elif self.lang == 'Tamil':
            self.max_len = 0
        else:
            raise Exception("Check Language")
        if self.mode == "train":
            self.path = 'Data/PartB_{}/Dev/'.format(self.lang)
        elif self.mode == "test":
            self.path = self.path = 'Data/PartB_{}/Dev/'.format(self.lang)
        else:
            raise Exception("Incorrect mode")
        self.file_list = os.listdir(os.path.join(self.path, 'Audio'))
        self.shuffle=shuffle
        self.csv_file = pd.read_csv(self.path + 'samples_450_{}.tsv'.format(client), header=None, sep='\t')
        self.input_length = []
        self.label_length = []

    def __len__(self):
        return len(self.csv_file)

    def pad(self, wav, trans, max_len):
        orig_len = len(wav)
        while len(wav) < max_len:
            diff = max_len - len(wav)
            ext = wav[:diff]
            wav = np.append(wav, wav[:diff])
            ratio = int(len(trans)*diff/len(wav))
            trans +=trans[:ratio]
        return wav, trans

    def preprocess(self, wav, sr, trans):

        out = stft(wav, win_length=int(sr*0.02), hop_length=int(sr*0.01))
        text_transform = TextTransform()
        trans = torch.Tensor(text_transform.text_to_int(trans.lower()))

        out = magphase(out)[0]
        out = [np.log(1 + x) for x in out]
        return np.array(out), trans

    def __getitem__(self, idx):
        file_name = self.csv_file[0][idx]
        trans = self.csv_file[1][idx]
        wav, sr = librosa.load(glob(self.path + 'Audio/*'+ str(file_name) + '.wav')[0])
        length_wav, length_trans = len(wav), len(trans)
        if len(set(trans)) > 2:
            label = 1
        elif len(set(trans)) == 1 or len(set(trans)) == 2:
            label = 0
        else:
            raise Exception("Check transcript")
            
        if self.mode =="train":
            return wav, sr, trans, self.lang
        elif self.mode == "test":
            return wav
        else:
            raise Exception("Incorrect Mode")
            

In [ ]:
class CodeSwitchDataset(Dataset):
    def __init__(self, lang, mode = "train", shuffle=True):
        self.mode = mode
        # data path
        self.lang = lang
        if self.lang == "Gujarati":
            self.max_len = 0
        elif self.lang == "Telugu":
            self.max_len = 529862
        elif self.lang == 'Tamil':
            self.max_len = 0
        else:
            raise Exception("Check Language")
        if self.mode == "train":
            self.path = 'Data/PartB_{}/Dev/'.format(self.lang)
        elif self.mode == "test":
            self.path = self.path = 'Data/PartB_{}/Dev/'.format(self.lang)
        else:
            raise Exception("Incorrect mode")
        self.file_list = os.listdir(os.path.join(self.path, 'Audio'))
        self.shuffle=shuffle
        self.csv_file = pd.read_csv(self.path + 'samples_450_b.tsv', header=None, sep='\t')
        self.input_length = []
        self.label_length = []

    def __len__(self):
        return len(self.csv_file)

    def pad(self, wav, trans, max_len):
        orig_len = len(wav)
        while len(wav) < max_len:
            diff = max_len - len(wav)
            ext = wav[:diff]
            wav = np.append(wav, wav[:diff])
            ratio = int(len(trans)*diff/len(wav))
            trans +=trans[:ratio]
        return wav, trans

    def preprocess(self, wav, sr, trans):

        out = stft(wav, win_length=int(sr*0.02), hop_length=int(sr*0.01))
        text_transform = TextTransform()
        trans = torch.Tensor(text_transform.text_to_int(trans.lower()))

        out = magphase(out)[0]
        out = [np.log(1 + x) for x in out]
        return np.array(out), trans

    def __getitem__(self, idx):
        file_name = self.csv_file[0][idx]
        trans = self.csv_file[1][idx]
        wav, sr = librosa.load(glob(self.path + 'Audio/*'+ str(file_name) + '.wav')[0])

        if len(set(trans)) > 2:
            label = 1
        elif len(set(trans)) == 1 or len(set(trans)) == 2:
            label = 0
        else:
            raise Exception("Check transcript")
        if self.mode =="train":
            return wav, sr, trans, self.lang
        elif self.mode == "test":
            return wav
        else:
            raise Exception("Incorrect Mode")


In [4]:
class TextTransform:
    """Maps characters to integers and vice versa"""
    def __init__(self):
        char_map_str = """
        s 1
        e 2
        t 3
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string)



In [5]:

train_audio_transforms = nn.Sequential(
            torchaudio.transforms.MelSpectrogram(n_mels=128, sample_rate = 22050, n_fft = 512, win_length=int(22050*0.02), hop_length=int(22050*0.01)),
            torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
            torchaudio.transforms.TimeMasking(time_mask_param=35)
        )


valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

text_transform = TextTransform()


def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _) in data:
        waveform=torch.Tensor(waveform)
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, padding_value = 1, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths


BATCH_SIZE = 4

def wer(r, h):
    """
    Calculation of WER with Levenshtein distance.

    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.

    Parameters
    ----------
    r : list
    h : list

    Returns
    -------
    int

    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation

    d = np.zeros((len(r) + 1) * (len(h) + 1), dtype=np.uint8)
    d = d.reshape((len(r) + 1, len(h) + 1))
    for i in range(len(r) + 1):
        for j in range(len(h) + 1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i
    for i in range(1, len(r) + 1):
        for j in range(1, len(h) + 1):
            if r[i - 1] == h[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
    return d[len(r)][len(h)]

def GreedyDecoder(output, labels, label_lengths, blank_label=0, collapse_repeated=True):
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets

In [6]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


class SpeechRecognitionModel(nn.Module):
    """Speech Recognition Model Inspired by DeepSpeech 2"""

    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 64, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[

            ResidualCNN(64, 64, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*64, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x

In [7]:
def train(model, device, _data, criterion, optimizer, epoch, iter_meter, writer, scheduler, client_no):
    model.train()
    total_loss = 0
    LR = 0
    train_loss = 0

    avg_acc = 0
    acc = []
    wers = []
    #bi, wav, label = batch_idx, wav, label
    for g in optimizer.param_groups:
        LR=g['lr']
    wav, labels, input_lengths, label_lengths = _data
    wav, labels = wav.to(device), labels.to(device)


    # input_lengths, label_lengths = torch.IntTensor(input_lengths), torch.IntTensor(label_lengths)
    
    # wav = wav.float()
    
    optimizer.zero_grad()
    # output = model(wav, input_lengths)   #(batch, time, n_class) [4, 911, 3]
    output = model(wav)

    output = F.log_softmax(output, dim=2)
    output = output.transpose(0,1)

    # print(labels, label_lengths)
    loss = criterion(output, labels, input_lengths, label_lengths)
    loss.backward()
    optimizer.step()
    scheduler.step()
    #print(loss)
    total_loss+=loss

    iter_meter.step()
    decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
    decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
    print(decoded_preds, decoded_targets)
    #print("preds: ", "".join(decoded_preds))
    for j in range(len(decoded_preds)):
        s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
        wers.append(wer(decoded_targets[j], decoded_preds[j]))
        acc.append(s.ratio())

    avg_acc = sum(acc)/len(acc)
    writer.add_scalar("{}/accuracy/train_accuracy".format(client_no), avg_acc, epoch)
    writer.add_scalar('{}/accuracy/train_loss'.format(client_no), loss.item(), iter_meter.get())
    writer.add_scalar('{}/CTCLoss'.format(client_no), loss, iter_meter.get())
    writer.add_scalar('{}/TLoss'.format(client_no), total_loss, iter_meter.get())
    writer.add_scalar("{}/Learning Rate".format(client_no), LR, epoch)

    writer.add_scalar("WER", wer(decoded_targets[j], decoded_preds[j]), iter_meter.get())
    """if batch_idx % 100 == 0 or batch_idx == data_len:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(wav), data_len,
                100. * batch_idx / len(train_loader), loss.item()))
        print("Train Accuracy: {}, Train loss: {}".format(avg_acc, train_loss))
    """

    #print(decoded_preds[0])
    """
    if (epoch+1)%2 == 0:
        model.eval().cpu()
        ckpt_model_filename = "{}_ckpt_epoch_".format(client_no) + str(epoch+1) + "_450a.pth"
        ckpt_model_path = os.path.join("Audio/MyDrive/checkpoints_avg/", ckpt_model_filename)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            }, ckpt_model_path)
        model.to(device).train()
    """
    return model

In [8]:
def test(model, device, _data, criterion, epoch, writer, client_no, iter_meter):
    model.eval()
    training_loss, train_acc = 0, 0
    eer, total_eer = 0, 0
    test_loss=0
    acc = []
    with torch.no_grad():
        inputs, labels, input_lengths, label_lengths = _data 
        inputs, labels = inputs.to(device), labels.to(device)
        # output = model(inputs, input_lengths)  # (batch, time, n_class)
        output=model(inputs)
        output = F.log_softmax(output, dim=2)

        output = output.transpose(0, 1) # (time, batch, n_class)
        loss = criterion(output, labels, input_lengths, label_lengths)
        test_loss += loss.item() / len(test_loader)
        decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
        decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
        for j in range(len(decoded_preds)):
            s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
            acc.append(s.ratio())

        avg_acc = sum(acc)/len(acc)
        writer.add_scalar("{}/test_accuracy".format(client_no), avg_acc, epoch)
        writer.add_scalar("{}/WER".format(client_no), wer(decoded_targets[j], decoded_preds[j]), iter_meter.get())
        writer.add_scalar('{}/test_loss'.format(client_no), test_loss, epoch)
        print("Test Accuracy: {}, Test loss: {}".format(avg_acc, test_loss))

In [9]:
def load_checkpoint(model, optimizer, filename='checkpoint.pth.tar'):
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.
    start_epoch = 1
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename, map_location = torch.device('cpu'))
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer, start_epoch


class IterMeter(object):
    """keeps track of total iterations"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val

In [10]:
device = 'cuda'

hparams = {
        "n_cnn_layers": 4,
        "n_rnn_layers": 5,
        "rnn_dim": 512,
        "n_class": 4,
        "n_feats": 128,
        "stride": 2,
        "dropout": 0.1,
        "learning_rate": 5e-4,
        "batch_size": 4,
        "epochs": 60
    }

In [11]:
LANGUAGE = "Telugu"

In [52]:
train_dataset_client1 = CodeSwitchDataset(lang=LANGUAGE, client = 'a', mode="train")
train_dataset_client2 = CodeSwitchDataset(lang=LANGUAGE, client = 'b', mode="train")
validation_split = 0.2
shuffle_dataset = True
random_seed = 42



dataset_size_1 = len(train_dataset_client1)
dataset_size_2 = len(train_dataset_client2)
indices_1 = list(range(dataset_size_1))
indices_2 = list(range(dataset_size_2))

split_1 = int(np.floor(validation_split * dataset_size_1))
split_2 = int(np.floor(validation_split * dataset_size_2))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices_1)
    np.random.shuffle(indices_2)
    
train_indices_1, val_indices_1 = indices_1[split_1:], indices_1[:split_1]
train_indices_2, val_indices_2 = indices_2[split_2:], indices_2[:split_2]
# Creating PT data samplers and loaders:
train_sampler_1 = SubsetRandomSampler(train_indices_1)
valid_sampler_1 = SubsetRandomSampler(val_indices_1)


train_sampler_2 = SubsetRandomSampler(train_indices_2)
valid_sampler_2 = SubsetRandomSampler(val_indices_2)



client1_train_loader = DataLoader(train_dataset_client1,
                          batch_size=BATCH_SIZE,
                          drop_last=True,
                          num_workers = 0,
                          sampler = train_sampler_1,
                          collate_fn = lambda x: data_processing(x, 'train'))


client2_train_loader = DataLoader(train_dataset_client2,
                          batch_size=BATCH_SIZE,
                          drop_last=True,
                          num_workers = 0,
                          sampler = train_sampler_2,
                          collate_fn = lambda x: data_processing(x, 'train'))



client1_test_loader = DataLoader(train_dataset_client1,
                          batch_size=BATCH_SIZE,
                          drop_last=True,
                          num_workers=0,
                          sampler=valid_sampler_1,
                          collate_fn=lambda x: data_processing(x, 'valid'))


client2_test_loader = DataLoader(train_dataset_client2,
                          batch_size=BATCH_SIZE,
                          drop_last=True,
                          num_workers=0,
                          sampler=valid_sampler_2,
                          collate_fn=lambda x: data_processing(x, 'valid'))

device = torch.device('cpu')
torch.cuda.empty_cache()

In [53]:
client1_model = SpeechRecognitionModel(
        hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
        hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
        ).to(device)
client1_optimizer = optim.Adam(client1_model.parameters(), hparams['learning_rate'])

client2_model = SpeechRecognitionModel(
        hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
        hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
        ).to(device)
client2_optimizer = optim.Adam(client2_model.parameters(), hparams['learning_rate'])


#client1_model, _, epoch_num = load_checkpoint(client1_model, client1_optimizer, "/content/Audio/MyDrive/PartB_{}/ckpt_epoch_36_batch_id_1645.pth".format(LANGUAGE))
#client2_model, _, epoch_num = load_checkpoint(client2_model, client2_optimizer, "/content/Audio/MyDrive/PartB_{}/ckpt_epoch_36_batch_id_1645.pth".format(LANGUAGE))

client1_model, _, epoch_num = load_checkpoint(client1_model, client1_optimizer, "Telugu_Checkpoints/ckpt_epoch_36_batch_id_1645.pth".format(LANGUAGE))
client2_model, _, epoch_num = load_checkpoint(client2_model, client2_optimizer, "Telugu_Checkpoints/ckpt_epoch_36_batch_id_1645.pth".format(LANGUAGE))

=> loading checkpoint 'Telugu_Checkpoints/ckpt_epoch_36_batch_id_1645.pth'
=> loaded checkpoint 'Telugu_Checkpoints/ckpt_epoch_36_batch_id_1645.pth' (epoch 35)
=> loading checkpoint 'Telugu_Checkpoints/ckpt_epoch_36_batch_id_1645.pth'
=> loaded checkpoint 'Telugu_Checkpoints/ckpt_epoch_36_batch_id_1645.pth' (epoch 35)


In [54]:
criterion = nn.CTCLoss(blank=0, reduction='mean').to(device)
epochs = 60
epoch_num = 1
client1_scheduler = optim.lr_scheduler.OneCycleLR(client1_optimizer,
    max_lr=hparams['learning_rate'],
    steps_per_epoch=int(len(client1_train_loader)),
    epochs=hparams['epochs'],
    anneal_strategy='linear')

client2_scheduler = optim.lr_scheduler.OneCycleLR(client2_optimizer,
    max_lr=hparams['learning_rate'],
    steps_per_epoch=int(len(client2_train_loader)),
    epochs=hparams['epochs'],
    anneal_strategy='linear')


#model, optimizer, epoch_num = load_checkpoint(model, optimizer, "checkpoints/ckpt_epoch_30_batch_id_1645.pth")

print(epoch_num)
writer = SummaryWriter('train_logs_450/')
iter_meter_client1 = IterMeter()
iter_meter_client2 = IterMeter()

1


In [55]:
"""
models = [client1_model, client2_model]
optimizers = [client1_optimizer, client2_optimizer]
train_loaders = [client1_train_loader, client2_train_loader]
schedulers = [client1_scheduler, client2_scheduler]
iter_meter = [iter_meter_client1, iter_meter_client2]
for epoch in range(1, epochs+1):
    print("----------Epoch: ", epoch, "----------------")
    _test_data = next(iter(test_loader))
    for i in range(len(models)):
        _train_data = next(iter(train_loaders[i]))
        #models[i].to(device)
        print("training: ", i)
        
        print("Train data :----------------------------", len(_train_data), _train_data[0].shape, _train_data)
        print("Test data :----------------------------", len(_test_data), _test_data[0].shape, _test_data)
        
        #models[i] = train(models[i], device, _train_data, criterion, optimizers[i], epoch, iter_meter[i], writer, schedulers[i], "client_{}".format(i))
        #test(models[i], device, _test_data, criterion, epoch, writer, "client_{}".format(i))
    #fed_model = federated_avg({'client1': models[0],
    #                          'client2': models[1]})
    #test(fed_model, device, _test_data, criterion, epoch, writer, "client_{}".format(i), iter_meter[i])
#fed_model.eval().cpu()
#save_model_filename = "final_epoch_client1" + str(epoch + 1)  + ".model"
#save_model_path = os.path.join("fed_checkpoints", save_model_filename)
#torch.save({
#            'epoch': epoch,
#            'model_state_dict': fed_model.state_dict(),
#            }, save_model_path)
#
#print("\nDone, trained model saved at", save_model_path)
"""

'\nmodels = [client1_model, client2_model]\noptimizers = [client1_optimizer, client2_optimizer]\ntrain_loaders = [client1_train_loader, client2_train_loader]\nschedulers = [client1_scheduler, client2_scheduler]\niter_meter = [iter_meter_client1, iter_meter_client2]\nfor epoch in range(1, epochs+1):\n    print("----------Epoch: ", epoch, "----------------")\n    _test_data = next(iter(test_loader))\n    for i in range(len(models)):\n        _train_data = next(iter(train_loaders[i]))\n        #models[i].to(device)\n        print("training: ", i)\n        \n        print("Train data :----------------------------", len(_train_data), _train_data[0].shape, _train_data)\n        print("Test data :----------------------------", len(_test_data), _test_data[0].shape, _test_data)\n        \n        #models[i] = train(models[i], device, _train_data, criterion, optimizers[i], epoch, iter_meter[i], writer, schedulers[i], "client_{}".format(i))\n        #test(models[i], device, _test_data, criterion,

In [56]:
client1_model.classifier.training

True

In [47]:
def train(model, device, train_loader, criterion, optimizer, epoch, iter_meter, writer, scheduler, ckpt_save_dir):
    model.train()
    data_len = len(train_loader.dataset)
    total_loss=0
    LR = 0
    train_loss = 0

    avg_acc = 0
    acc = []
    wers = []
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for batch_idx, (_data) in enumerate(train_loader):
        print("------------------ITER------------------")
        #bi, wav, label = batch_idx, wav, label
        for g in optimizer.param_groups:
            LR=g['lr']
        wav, labels, input_lengths, label_lengths = _data
        # input_lengths, label_lengths = torch.IntTensor(input_lengths), torch.IntTensor(label_lengths)
        wav = wav.to(device)
        # wav = wav.float()
        labels = labels.to(device)
        optimizer.zero_grad()
        # output = model(wav, input_lengths)   #(batch, time, n_class) [4, 911, 3]
        output = model(wav)

        output = F.log_softmax(output, dim=2)


        output = output.transpose(0,1)
        
        # print(labels, label_lengths)
        loss = criterion(output, labels, input_lengths, label_lengths)
        loss.backward()
        optimizer.step()
        scheduler.step()
        #print(loss)
        total_loss+=loss

        iter_meter.step()
        decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
        decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
        print(decoded_preds, decoded_targets)
        print("preds: ", "".join(decoded_preds))
        for j in range(len(decoded_preds)):
            s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
            wers.append(wer(decoded_targets[j], decoded_preds[j]))
            acc.append(s.ratio())

        avg_acc = sum(acc)/len(acc)
        writer.add_scalar("accuracy/train_accuracy", avg_acc, epoch)
        writer.add_scalar('accuracy/train_loss', loss.item(), iter_meter.get())
        writer.add_scalar('CTCLoss', loss, epoch*len(train_loader)+1)
        writer.add_scalar('TLoss', total_loss, epoch*len(train_loader)+1)
        writer.add_scalar("Learning Rate", LR, epoch)

        writer.add_scalar("WER", wer(decoded_targets[j], decoded_preds[j]), iter_meter.get())
        if batch_idx % 100 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(wav), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))
            print("Train Accuracy: {}, Train loss: {}".format(avg_acc, train_loss))
        model.to(device).train()
    #return model, optimizer
    # for g in optimizer.param_groups:
    #   g['lr'] = g['lr']/LEARNING_ANNEAL
    """
    #print(decoded_preds[0])
    if (epoch+1)%2 == 0:
        model.eval().cpu()
        ckpt_model_filename = "ckpt_epoch_" + str(epoch+1) + "_batch_id_" + str(batch_idx+1) + ".pth"
        ckpt_model_path = os.path.join(ckpt_save_dir, ckpt_model_filename)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            }, ckpt_model_path)
        model.to(device).train()
    """

In [48]:
def test(model, device, test_loader, criterion, epoch, writer):
    model.eval()
    training_loss, train_acc = 0, 0
    eer, total_eer = 0, 0
    test_loss=0
    acc = []
    with torch.no_grad():
        for batch_idx, _data in enumerate(test_loader):
            inputs, labels, input_lengths, label_lengths = _data 
            inputs, labels = inputs.to(device), labels.to(device)
            # output = model(inputs, input_lengths)  # (batch, time, n_class)
            output=model(inputs)
            output = F.log_softmax(output, dim=2)
            
            output = output.transpose(0, 1) # (time, batch, n_class)
            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)
            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
            decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
            for j in range(len(decoded_preds)):
                s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
                acc.append(s.ratio())

            avg_acc = sum(acc)/len(acc)
            writer.add_scalar("test_accuracy", avg_acc, epoch)
            writer.add_scalar('test_loss', test_loss, epoch)
            print("Test Accuracy: {}, Test loss: {}".format(avg_acc, test_loss))
        

In [49]:
for name, param in client1_model.state_dict().items():
    print(name, param.size())
    print(param)

cnn.weight torch.Size([64, 1, 3, 3])
tensor([[[[ 2.0512e-01,  7.2427e-02,  6.2728e-02],
          [ 4.3884e-02, -8.3980e-02, -3.0685e-02],
          [ 1.2385e-01, -2.1300e-01, -8.8035e-02]]],


        [[[-2.5716e-01,  6.3218e-02,  2.3704e-01],
          [-3.9578e-01,  1.0188e-01,  9.4281e-02],
          [-3.2622e-02,  5.7981e-02, -4.4567e-02]]],


        [[[ 4.3063e-01,  3.5707e-01,  2.2267e-01],
          [ 7.7468e-01,  7.0123e-01, -7.3938e-02],
          [ 9.9112e-02,  5.4483e-01,  2.7855e-01]]],


        [[[ 9.7231e-02,  4.1032e-01,  1.7382e-01],
          [-3.0798e-01,  3.0852e-01,  5.9512e-01],
          [-4.6495e-01, -2.5291e-01, -2.0049e-01]]],


        [[[ 6.7359e-02,  1.5889e-02,  4.0100e-01],
          [-3.4750e-02, -3.7033e-01, -5.0606e-02],
          [-4.3513e-01,  9.8779e-02,  1.2856e-01]]],


        [[[-2.6491e-01,  2.4393e-01, -1.3148e-01],
          [ 3.3702e-02,  1.6991e-01,  1.7112e-01],
          [-1.0562e-01,  3.6429e-01,  9.3047e-03]]],


        [[[-1.8632e-0

In [51]:
for name, param in client1_model.state_dict().items():
    print(name, param.size())
    print(param)

cnn.weight torch.Size([64, 1, 3, 3])
tensor([[[[ 2.0512e-01,  7.2431e-02,  6.2728e-02],
          [ 4.3865e-02, -8.3963e-02, -3.0666e-02],
          [ 1.2385e-01, -2.1298e-01, -8.8034e-02]]],


        [[[-2.5720e-01,  6.3179e-02,  2.3700e-01],
          [-3.9580e-01,  1.0185e-01,  9.4279e-02],
          [-3.2643e-02,  5.7972e-02, -4.4562e-02]]],


        [[[ 4.3066e-01,  3.5706e-01,  2.2272e-01],
          [ 7.7470e-01,  7.0127e-01, -7.3897e-02],
          [ 9.9122e-02,  5.4484e-01,  2.7862e-01]]],


        [[[ 9.7211e-02,  4.1028e-01,  1.7376e-01],
          [-3.0796e-01,  3.0852e-01,  5.9514e-01],
          [-4.6496e-01, -2.5293e-01, -2.0051e-01]]],


        [[[ 6.7365e-02,  1.5878e-02,  4.0099e-01],
          [-3.4755e-02, -3.7033e-01, -5.0605e-02],
          [-4.3512e-01,  9.8792e-02,  1.2856e-01]]],


        [[[-2.6495e-01,  2.4388e-01, -1.3144e-01],
          [ 3.3662e-02,  1.6983e-01,  1.7115e-01],
          [-1.0561e-01,  3.6429e-01,  9.3323e-03]]],


        [[[-1.8630e-0

tensor([[-0.0390, -0.0180, -0.0041,  ..., -0.0037,  0.0099,  0.0460],
        [ 0.0344, -0.0040, -0.0207,  ..., -0.0316,  0.0018,  0.0096],
        [-0.0154,  0.0266,  0.0404,  ...,  0.0055,  0.0274,  0.0553],
        [-0.0044, -0.0151,  0.0124,  ...,  0.0031,  0.0157,  0.0511]])
classifier.3.bias torch.Size([4])
tensor([ 0.3474, -0.1841, -0.2395, -0.0883])


In [50]:
for epoch in range(1, epochs+1):
    print("-----------------EPOCH: {}-----------------".format(epoch))
    print(client1_model.classifier[3].weight.grad)
    client1_model, client1_optimizer = train(client1_model, device, client1_train_loader, criterion, client1_optimizer, epoch, iter_meter_client1, writer, client1_scheduler, "fed_averaging")
    print(client1_model.classifier[3].weight.grad)
    test(client1_model, device, client1_test_loader, criterion, epoch, writer)

    client2_model, client2_optimizer = train(client2_model, device, client2_train_loader, criterion, client2_optimizer, epoch, iter_meter_client2, writer, client2_scheduler, "fed_averaging")
    test(client2_model, device, client2_test_loader, criterion, epoch, writer)

    fed_model = federated_avg({'client1': client1_model,
                              'client2': client2_model})
    test(fed_model, device, client1_test_loader, criterion, epoch, writer)
    
fed_model.eval().cpu()
save_model_filename = "final_epoch_client1" + str(epoch + 1)  + ".model"
save_model_path = os.path.join("fed_checkpoints", save_model_filename)
torch.save({
            'epoch': epoch,
            'model_state_dict': fed_model.state_dict(),
            }, save_model_path)

print("\nDone, trained model saved at", save_model_path)





  0%|          | 0/90 [00:00<?, ?it/s]

-----------------EPOCH: 1-----------------
tensor([[-1.7061e-07, -4.2971e-08, -1.8999e-08,  ..., -5.1188e-09,
         -6.5060e-09, -2.7314e-07],
        [ 1.8045e-07,  1.4972e-09,  2.6546e-08,  ...,  6.8954e-09,
          7.8316e-09,  2.6940e-07],
        [-3.1275e-08, -9.4515e-10, -1.0075e-08,  ..., -9.0712e-09,
         -1.7137e-08, -1.2328e-08],
        [ 2.1434e-08,  4.2419e-08,  2.5278e-09,  ...,  7.2947e-09,
          1.5811e-08,  1.6067e-08]])
------------------ITER------------------
['sssss', 'ssss', 'ssssss', 'ssssss'] ['sstteeettttttttttttsssssstttttts', 'ssttettttetttttttttttttts', 'sssstettttttttttttstttttttttttttttttttttttttss', 'ssssttttttttteeeeeeeeeettssssseeettsettttttts']
preds:  sssssssssssssssssssss
Train Epoch: 1 [0/450 (0%)]	Loss: 0.389539
Train Accuracy: 0.23580754260267445, Train loss: 0
------------------ITER------------------
['ssss', 'sstss', 'ssss', 'ssss'] ['ssstttttttttttttssttteetttttts', 'sstttttttttteetttss', 'ssstttttssssssssttttttttetttss', 'sstttttt

KeyboardInterrupt: 

In [39]:
def train(model, device, train_loader, criterion, optimizer, epoch, iter_meter, writer, scheduler, ckpt_save_dir):
    model.train()
    data_len = len(train_loader.dataset)
    total_loss=0
    LR = 0
    train_loss = 0

    avg_acc = 0
    acc = []
    wers = []
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for batch_idx, (_data) in enumerate(train_loader):
        print("------------------ITER------------------")
        #bi, wav, label = batch_idx, wav, label
        for g in optimizer.param_groups:
            LR=g['lr']
        wav, labels, input_lengths, label_lengths = _data
        # input_lengths, label_lengths = torch.IntTensor(input_lengths), torch.IntTensor(label_lengths)
        wav = wav.to(device)
        # wav = wav.float()
        labels = labels.to(device)
        optimizer.zero_grad()
        # output = model(wav, input_lengths)   #(batch, time, n_class) [4, 911, 3]
        output = model(wav)

        output = F.log_softmax(output, dim=2)


        output = output.transpose(0,1)
        
        # print(labels, label_lengths)
        loss = criterion(output, labels, input_lengths, label_lengths)
        loss.backward()
        optimizer.step()
        scheduler.step()
        #print(loss)
        total_loss+=loss

        iter_meter.step()
        decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
        decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
        print(decoded_preds, decoded_targets)
        print("preds: ", "".join(decoded_preds))
        for j in range(len(decoded_preds)):
            s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
            wers.append(wer(decoded_targets[j], decoded_preds[j]))
            acc.append(s.ratio())

        avg_acc = sum(acc)/len(acc)
        writer.add_scalar("accuracy/train_accuracy", avg_acc, epoch)
        writer.add_scalar('accuracy/train_loss', loss.item(), iter_meter.get())
        writer.add_scalar('CTCLoss', loss, epoch*len(train_loader)+1)
        writer.add_scalar('TLoss', total_loss, epoch*len(train_loader)+1)
        writer.add_scalar("Learning Rate", LR, epoch)

        writer.add_scalar("WER", wer(decoded_targets[j], decoded_preds[j]), iter_meter.get())
        if batch_idx % 100 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(wav), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))
            print("Train Accuracy: {}, Train loss: {}".format(avg_acc, train_loss))
        model.to(device).train()
    return model, optimizer
    

In [22]:
for epoch in range(1, epochs+1):
    print("-----------------EPOCH: {}-----------------".format(epoch))
    print(client1_model.classifier[3].weight.grad)
    client1_model, client1_optimizer = train(client1_model, device, client1_train_loader, criterion, client1_optimizer, epoch, iter_meter_client1, writer, client1_scheduler, "fed_averaging")
    print(client1_model.classifier[3].weight.grad)
    test(client1_model, device, client1_test_loader, criterion, epoch, writer)

    client2_model, client2_optimizer = train(client2_model, device, client2_train_loader, criterion, client2_optimizer, epoch, iter_meter_client2, writer, client2_scheduler, "fed_averaging")
    test(client2_model, device, client2_test_loader, criterion, epoch, writer)

    fed_model = federated_avg({'client1': client1_model,
                              'client2': client2_model})
    test(fed_model, device, client1_test_loader, criterion, epoch, writer)
    


-------------EPOCH-----------


  0%|          | 0/90 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:

for epoch in range(1, epochs+1):
    _test_data = next(iter(test_loader))
    print(_test_data)
    for i in range(len(models)):
        _train_data = next(iter(train_loaders[i]))
        print("training: ", i)
        

In [35]:
_train_data

(tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [3.4077e-04, 7.0382e-05, 1.9882e-05,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [9.2415e-04, 1.9087e-04, 5.3919e-05,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           ...,
           [9.5825e-08, 1.4270e-16, 1.1959e-16,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [9.5467e-08, 1.2667e-16, 1.0774e-16,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [9.6489e-08, 1.1667e-16, 7.9653e-17,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00]]],
 
 
         [[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [1.2004e-05, 6.4693e-04, 9.3381e-04,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [3.2553e-05, 1.7544e-03, 2.5324e-03,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           ...,
           [7.2002e-07, 1.6514e-15,

In [37]:
_train_data

(tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [1.6102e-02, 2.5319e-03, 7.1354e-04,  ..., 3.8064e-03,
            1.5619e-02, 1.0263e-02],
           [4.3668e-02, 6.8663e-03, 1.9351e-03,  ..., 1.0323e-02,
            4.2357e-02, 2.7834e-02],
           ...,
           [4.2098e-06, 3.6487e-15, 9.8787e-15,  ..., 2.3212e-14,
            1.9503e-14, 1.3838e-07],
           [4.1987e-06, 1.8435e-15, 6.9616e-15,  ..., 1.9385e-14,
            1.5223e-14, 1.4079e-07],
           [4.2461e-06, 1.4643e-15, 7.0396e-15,  ..., 1.4656e-14,
            1.4816e-14, 1.4411e-07]]],
 
 
         [[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [2.3314e-05, 4.7871e-05, 1.8838e-04,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [6.3227e-05, 1.2982e-04, 5.1088e-04,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           ...,
           [0.0000e+00, 0.0000e+00,